# Azure ML Hyperparameter Search Pipeline for COVID-CXR
This notebook defines an Azure machine learning pipeline for a hyperparameter search and submits the pipeline as an experiment to be run on an Azure virtual machine.

In [ ]:
# Import statements
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace, Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep, HyperDriveStep, HyperDriveStepRun
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, uniform, loguniform
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails
import shutil
import math

In [ ]:
# Define some constants
CT_NAME = "nd12s-clust1"            # Name of our compute cluster
VM_SIZE = "STANDARD_ND12S"          # Specify the Azure VM for execution of our pipeline
MIN_NODES = 10
MAX_NODES = 10                      # Max number of compute nodes in cluster
TOTAL_RUNS = 100                    # Total training runs in hyperparameter search
MAX_DURATION_MINUTES = 12 * 60      # Max # minutes to run the experiment for. (# hours * 60 min/hour)
PRIMARY_METRIC = 'validation_auc'   # Primary metric for optimization in hyperparameter search
WARM_START_RUNS = []                # List of HyperDriveRuns already ran to guide this experiment

### Register the workspace and configure its Python environment.

In [ ]:
# Get reference to the workspace
ws = Workspace.from_config("./ws_config.json")

# Set workspace's environment
env = Environment.from_pip_requirements(name = "covid-cxr_env", file_path = "./../requirements.txt")
env.register(workspace=ws)
runconfig = RunConfiguration(conda_dependencies=env.python.conda_dependencies)
print(env.python.conda_dependencies.serialize_to_string())

# Move AML ignore file to root folder
aml_ignore_path = shutil.copy('./.amlignore', './../.amlignore') 

### Create references to persistent and intermediate data
Create DataReference objects that point to our raw data on the blob. Configure a PipelineData object to point to preprocessed images stored on the blob.

In [ ]:
# Get the blob datastore associated with this workspace
blob_store = Datastore(ws, name='covid_cxr_ds')

# Create data references to folders on the blob
raw_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="raw_data",
    path_on_datastore="data/")
mila_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="mila_data",
    path_on_datastore="data/covid-chestxray-dataset/")
fig1_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="fig1_data",
    path_on_datastore="data/Figure1-COVID-chestxray-dataset/")
rsna_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="rsna_data",
    path_on_datastore="data/rsna/")

# Set up references to pipeline data (intermediate pipeline storage).
processed_pd = PipelineData(
    "processed_data",
    datastore=blob_store,
    output_name="processed_data")
metrics_pd = PipelineData(
    name='hparam_metrics_data',
    datastore=blob_store,
    pipeline_output_name="hparam_metric_data")

### Compute Target
Specify and configure the compute target for this workspace. If a compute cluster by the name we specified does not exist, create a new compute cluster.

In [ ]:
# Set up the compute target for this
try:
    compute_target = AmlCompute(ws, CT_NAME)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating new compute target")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                                min_nodes=MIN_NODES, 
                                                                max_nodes=MAX_NODES)    
    compute_target = ComputeTarget.create(ws, CT_NAME, provisioning_config)  # Create the compute cluster
    
    # Wait for cluster to be provisioned
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20) 
    
print("Azure Machine Learning Compute attached")
print("Compute targets: ", ws.compute_targets)
compute_target = ws.compute_targets[CT_NAME]

### Configure hyperparameter search experiment
We will launch multiple runs on out compute cluster to run our hyperparameter search. Below we define the ranges over which to search for hyperparameters. We will randomly sample over the defined range and pass the samples to each training run as arguments to the training script. Then we specify the primary metric to optimize. Finally, we specify a termination policy, to prevent resource wastage during poorly performing runs.

In [ ]:
# Define random sampling ranges
param_sampling = RandomParameterSampling( {
        "KERNEL_SIZE": choice('(3,3)', '(5,5)'), 
        "MAXPOOL_SIZE": choice('(2,2)', '(3,3)'),
        "INIT_FILTERS": choice(8, 16, 32),
        "FILTER_EXP_BASE": choice(range(2, 3)),
        "CONV_BLOCKS": choice(range(3, 9)),
        "NODES_DENSE0": choice(128, 256, 512, 1024),
        "LR": loguniform(math.log(1e-5), math.log(1e-3)),
        "OPTIMIZER": choice('adam'),
        "DROPOUT": choice(0.0, 0.1, 0.2, 0.3, 0.4, 0.5),
        "L2_LAMBDA": choice(0.0, 0.00001, 0.0001, 0.001),
    }
)

# Specify how we would like to optimize our primary metric
primary_metric_goal = PrimaryMetricGoal.MINIMIZE if 'loss' in PRIMARY_METRIC else PrimaryMetricGoal.MAXIMIZE

# Set termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=3, delay_evaluation=10)

### Define pipeline and submit experiment.
Define the steps of an Azure machine learning pipeline. Create an Azure Experiment that will run our pipeline. Submit the experiment to the execution environment.

In [ ]:
# Define preprocessing step the ML pipeline
step1 = PythonScriptStep(name="preprocess_step",
                         script_name="azure/preprocess_step/preprocess_step.py",
                         arguments=["--miladatadir", mila_data_dr, "--fig1datadir", fig1_data_dr, 
                                    "--rsnadatadir", rsna_data_dr, "--preprocesseddir", processed_pd],
                         inputs=[mila_data_dr, fig1_data_dr, rsna_data_dr],
                         outputs=[processed_pd],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         allow_reuse=True)

# Define hyperparameter search step in the ML pipeline
est = TensorFlow(source_directory='./../',
                   script_params=None,
                   compute_target=compute_target,
                   entry_script='azure/hparam_train_step/hparam_train_step.py',
                   pip_packages=['tensorboard', 'pandas', 'dill', 'numpy', 'imblearn', 'matplotlib', 'scikit-image', 'matplotlib',
                                'pydicom', 'opencv-python', 'tqdm', 'scikit-learn'],
                   use_gpu=True,
                   framework_version='2.0')
hd_config = HyperDriveConfig(estimator=est, 
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name=PRIMARY_METRIC, 
                             primary_metric_goal=primary_metric_goal, 
                             max_total_runs=TOTAL_RUNS,
                             max_concurrent_runs=MAX_NODES,
                             max_duration_minutes=MAX_DURATION_MINUTES,
                             resume_from=WARM_START_RUNS)
step2 = HyperDriveStep(name="hyperdrive_step",
                       hyperdrive_config=hd_config,
                       estimator_entry_script_arguments=["--rawdatadir", raw_data_dr, "--preprocesseddir", processed_pd],
                       inputs=[raw_data_dr, processed_pd],
                       metrics_output=metrics_pd)

# Construct the ML pipeline from the steps
steps = [step1, step2]
hparams_pipeline = Pipeline(workspace=ws, steps=steps)
hparams_pipeline.validate()

# Define a new experiment and submit a new pipeline run to the compute target.
experiment = Experiment(workspace=ws, name='HyperDriveExperiment_v1')
hyperdrive_run = experiment.submit(hparams_pipeline, regenerate_outputs=False)
print("HyperDrive pipeline is submitted for execution")

# Move AML ignore file back to original folder
aml_ignore_path = shutil.move(aml_ignore_path, './.amlignore')

### Retrieve results
Get the metrics of the pipeline run and download them to the local project. Then get the file name of the model with the best results

In [ ]:
# Wait for the pipeline to finish running.
hyperdrive_run.wait_for_completion()

# Use the Azure RunDetails widget to view results of the hyperparameter search
hd_step_run = HyperDriveStepRun(step_run=hyperdrive_run.find_step_run("hyperdrive_step")[0])
RunDetails(hd_step_run).show()
best_run = hd_step_run.get_best_run_by_primary_metric()

# Download the metrics from the hyperparameter search experiment.
metrics_output = hyperdrive_run.get_pipeline_output("hparam_metric_data")
num_file_downloaded = metrics_output.download('./../results/logs/hparam_search/', show_progress=True)

# Print all metrics from the best run
best_run_metrics = best_run.get_metrics()
print("Best model's metrics:")
for metric_name in best_run_metrics:
    print(str(metric_name) + ': ' + str(best_run_metrics[metric_name]))